In [1]:
devtools::install_github("chansigit/scFly", force=TRUE)
library("scFly")

✔  checking for file ‘/tmp/RtmpPv6mgx/remotes1b1b41bf50c84/chansigit-scFly-1fac931/DESCRIPTION’ (383ms)
─  preparing ‘scFly’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘scFly_0.1.0.tar.gz’ (1.1s)
   


# MNIST dataset loading

In [39]:
# Load mnist datasets
data(mnist.train.x)
data(mnist.train.y)
data(mnist.test.x)
data(mnist.test.y)

# Assign rownames/names for the matrices/vectors, this is required for we use them to get elements
rownames(mnist.train.x)<-1:nrow(mnist.train.x)
names(mnist.train.y)   <- 1:length(mnist.train.y)
rownames(mnist.test.x) <-1:nrow(mnist.test.x)
names(mnist.test.y)    <- 1:length(mnist.test.y)

# We choose a small subset as an example
query      <- mnist.train.x[1:100,]
query.label<- mnist.train.y[1:100]
reference      <- mnist.test.x[1:5000,]
reference.label<- mnist.test.y[1:5000]

dim(query)
dim(reference)

[1] 100 784

[1] 5000  784

In [40]:
head(query)
head(query.label)

0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


1 2 3 4 5 6 
5 0 4 1 9 2

# Build DenseFly embedding 

Embedding the vectors in the matrices into a dense binary vector form

`hash.length`-long short binary vectors are calculated first to pre-select similar items.

Long embedded vector with `nProj`*`hash.length` bits are then used for similarity comparison and nearest neighbour searching.

In [44]:
queryObj      <-BuildEmbeddedObject(data=query,     nProj=500, hash.length=10, sampling.rate=0.3, do.center=TRUE)
referenceObj  <-BuildEmbeddedObject(data=reference, nProj=500, hash.length=10, sampling.rate=0.3, do.center=TRUE)

Calculating activations and the binary codes takes 0.31 seconds.
100 data items are processed
Calculating activations and the binary codes takes 0.85 seconds.
5000 data items are processed


In [45]:
library("tictoc")
tic("NN searching")
FindAllNearestNeighbour(query=queryObj,            # embedded query object
                        reference = referenceObj,  # embedded reference object
                        topN=5,                    # for each query element, find 5 most similar elements from the reference set
                        ncores = 1)-> mapping.result
toc()

NN searching: 14.539 sec elapsed


# Results evaluation

In [46]:
mapping.result["query.label"]<-query.label[mapping.result$"query"]

Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}


for (i in 1:nrow(mapping.result)){
    reference.label[
        c(mapping.result[i,"nn_rowid1"], mapping.result[i,"nn_rowid2"], 
          mapping.result[i,"nn_rowid3"], mapping.result[i,"nn_rowid4"], 
          mapping.result[i,"nn_rowid5"])] -> prediction
    Mode(prediction) -> prediction
    
    mapping.result[i, "prediction"]<- prediction
    
}

sum(mapping.result$query.label==mapping.result$prediction)/nrow(mapping.result)

[1] 0.93